In [1]:
import csv
import json
import pandas as pd


/Users/jacopogneri/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/jacopogneri/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
json_files = ["comments-user-toxicity-2020-04", "comments-user-toxicity-2020-05", "comments-user-toxicity-2020-06", "comments-user-toxicity-2020-07", "comments-user-toxicity-2020-08", "comments-user-toxicity-2020-09", "comments-user-toxicity-2020-10", "comments-user-toxicity-2020-11", "comments-user-toxicity-2020-12", "comments-user-toxicity-2021-01"] 
folder = "../JSON_comments/"

In [3]:
%%time
lines_def = []
for i in range(0, len(json_files)):
    with open(folder+json_files[i], 'r') as incsv:
        for line in incsv:
            lines_def.append(json.loads(line))

CPU times: user 2min 56s, sys: 4min 5s, total: 7min 2s
Wall time: 8min 4s


In [4]:
len(lines_def)

8766596

# emoji analysis

In [5]:
import emoji
import emojis

In [6]:
# Define the function to extract emoji information (number, set and list of emojis)
def extract_emojis(listLine):
    for i in listLine:
        count_e = emoji.emoji_count(i["text"])
        #if count_e != 0:
         #   print(count_e)
        set_e = ''.join(emoji.distinct_emoji_list(i["text"]))
        data_list_e = emoji.emoji_list(i["text"])
        if data_list_e:
            str_e = ''.join([entry['emoji'] for entry in data_list_e])
        else:
            str_e = ''

        standard_s = emojis.decode(i["text"]) # the emojis are decoded (e.g. the smiley face becomes :smile:)
        #standard_s.replace(':','') #used to remove the colons
        i["text"] = standard_s
        i["emoji_count"] = count_e
        i["emoji_unique"] = set_e
        i["emoji_list"] = str_e


# Time handling

In [7]:
from datetime import datetime

In [8]:
#this function takes the key time, converts it into date and time_of day, adds these two keys and then removes the key time
def convert_time(listLine):
    for i in listLine:
        date = datetime.fromtimestamp(i["time"])
        i["date"] = date.date().strftime("%Y-%m-%d")
        i["time_of_day"] = date.time().strftime("%H:%M:%S")
        del i["time"]

# Text cleaning

In [9]:
from html import unescape
import re
import string
import contractions

In [10]:
# Define a function to clean a single text
def clean_single_text(listLine):
    for i in listLine:
      # Decode HTML entities
      i["text"] = unescape(i["text"])

      # Remove URLs
      i["text"] = re.sub(r'https?://\S+|www\.\S+', '', i["text"])

      # Remove symbols excluding numbers and punctuation
      i["text"] = re.sub(r'[^a-zA-Z0-9\s' + re.escape(string.punctuation) + ']', '', i["text"])

      # Reduce multiple spaces to one
      i["text"] = re.sub(r'\s+', ' ', i["text"])

      # Remove new lines and tabs
      i["text"] = re.sub(r'[\n\t]', ' ', i["text"])

      i["text"] = i["text"].strip()  # Remove leading and trailing spaces
    

In [11]:
#used to expand contraction (it's => it is). slang is set to True as it is possible to find slang words in reddit posts
#VERY FAST!
def expand_contractions(listLine):
    for i in listLine:
        i["text"] = contractions.fix(i["text"], slang=True)

In [12]:
def unique_upper_words(listLine):
    for i in listLine:
        # number of unique words in the text
        i["num_unique_words"] = len(set(str(i['text']).lower().split()))

        # number of Upper case words in the text
        i["num_words_upper"] = len([w for w in str(i["text"]).split() if w.isupper()])

In [13]:

my_punct = ['"', '$', '%', '&', "'", '(', ')', '*', '+', ',',
           '/', ':', ';', '<', '=', '>', '@', '[', '\\', ']', '^', '_',
           '`', '{', '|', '}', '~', '»', '«', '“', '”', '#', '!', '?','.',':']

punct_pattern = re.compile("[" + re.escape("".join(my_punct)) + "]")
#function that removes punctuation
def remove_punct(listLine):
    for i in listLine:
        i["text"] = re.sub(punct_pattern, ' ', i["text"])
        i["text"] = i["text"].lower()


# PosTag (non usa spacy)

In [15]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jacopogneri/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jacopogneri/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [18]:
#POS Tagging is useful to determine the number of "full words" (Verbs, Nouns and Adj) in a post
def posTag(listLine):
    for i in listLine:
        tokens = word_tokenize(i["text"])
        pos_tags = nltk.pos_tag(tokens)
        # Counting the full words (Adjectives, Nouns, Verbs)
        number_full_words = sum(1 for _, tag in pos_tags if tag.startswith(("JJ", "NN", "VB")))
        i["number_full_words"] = number_full_words


# NRCLex

In [19]:
from nrclex import NRCLex

In [20]:
# Define a function to extract emotion vectors using NRCLex
def get_emotion_vector(listLine):
    for i in listLine:
        emotion = NRCLex(i["text"])
        affect_frequencies_dict = {emotion_class: round(frequency, 2) for emotion_class, frequency in emotion.affect_frequencies.items()}
        for k, v in affect_frequencies_dict.items():
            i[k] = v


## Readability

In [ ]:
from readability import Readability
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

In [30]:
def readability_tests(listLine):
    for i in listLine:
        text_str = ''.join(i["text"])
        i["flesch_kincaid"] = 0
        i["flesch"] = 0
        i["fog"] = 0
        i["coleman_liau"] = 0
        i["dale_chall"] = 0
        i["ari"] = 0
        i["linsear_write"] = 0
        i["smog"] = 0
        i["spache"] = 0
        try:
            r = Readability(text_str)
            sentences = sent_tokenize(text_str)
            # Check if the sentence count is less than 30 for SMOG calculation
            num_sentences = len(sentences)

            # Tokenize words using nltk for flesch_kincaid()
            wordlst = text_str.split()
            num_words = len(wordlst)
            if num_words >= 100:
                i["flesch_kincaid"] = r.flesch_kincaid().score
                i["flesch"] = r.flesch().score
                i["fog"] = r.gunning_fog().score
                i["coleman_liau"] = r.coleman_liau().score
                i["dale_chall"] = r.dale_chall().score
                i["ari"] =r.ari().score
                i["linsear_write"] =r.linsear_write().score
                if num_sentences >= 30:
                    i["smog"] =r.smog().score 
                i["spache"] =r.spache().score
        except:
            continue
        
        

## TextBlob

In [34]:
from textblob import TextBlob

In [37]:
def get_polarity(listLine):
    for i in listLine:
        try:
            blob = TextBlob(i["text"])
            i["polarity"] = blob.sentiment.polarity
        except:
            i["polarity"] = 0

## VAD

In [23]:
VAD = pd.read_csv('../NRC/NRC-VAD-Lexicon.txt', sep="\t", header=None)
VAD.columns = ["word", "valence", "arousal", "dominance"]
VAD_dict = VAD.set_index('word').T.to_dict()

In [24]:
def emotion_VAD(text, dim):
    words_VAD = text.split()
    score = [VAD_dict[i][dim] if i in VAD_dict else 0 for i in words_VAD]
    return sum(score) / max(len(score), 1)

def analyze_valence(text):
    return emotion_VAD(text, 'valence')

def analyze_arousal(text):
    return emotion_VAD(text, 'arousal')

def analyze_dominance(text):
    return emotion_VAD(text, 'dominance')

In [25]:
def get_VAD(listLine):
    for i in listLine:
        i["valence"] = analyze_valence(i["text"])
        i["arousal"] = analyze_arousal(i["text"])
        i["dominance"] = analyze_dominance(i["text"])

## Apply functions

applico qui tutte le funzioni

In [ ]:
#EMOJI
extract_emojis(lines_def)
#TIME HANDLING
convert_time(lines_def)
#CLEAN TEXT
clean_single_text(lines_def)
#CONTRACTIONS
expand_contractions(lines_def)
#UPPER WORDS
unique_upper_words(lines_def)
#PUNCTUATION
remove_punct(lines_def)
#FULL WORDS
posTag(lines_def)
#EMOTIONS
get_emotion_vector(lines_def)
#READABILITY
readability_tests(lines_def)
#TEXTBLOB POLARITY
get_polarity(lines_def)
#VAD LEXICON
get_VAD(lines_def)


## Turn into dataset

In [40]:
df = pd.DataFrame.from_dict(lines_def, orient='columns')

In [41]:
df.to_csv('PIANO_comments.csv', index=False)